In [1]:
import os
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

intents_path = '/content/drive/MyDrive/Colab Notebooks/chatbot-flask-simple/data/intents'

# inference model variables
#inference_load_model_from = os.path.join(intents_path, 'intents_chatbot_model.h5')
inference_load_intents_from = os.path.join(intents_path, 'intents_job_intents.json')
#inference_load_classes = os.path.join(intents_path, 'intents_classes.pkl')
#inference_load_words = os.path.join(intents_path, 'intents_words.pkl')
#!cp intents_path .

words = []
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open(inference_load_intents_from, encoding='cp1252').read()
intents = json.loads(data_file)

for intent in intents['intents']:
     for pattern in intent['patterns']:
          w = nltk.word_tokenize(pattern)
          words.extend(w)

          documents.append((w, intent['tag']))

          if intent['tag'] not in classes:
               classes.append(intent['tag'])


words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print(len(documents), 'documents')
print(len(classes), 'classes', classes)
print(len(words), 'unique lemmatized words', words)

pickle.dump(words,open(os.path.join(intents_path, 'intents_words.pkl'),'wb'))
pickle.dump(classes,open(os.path.join(intents_path, 'intents_classes.pkl'),'wb'))

# init training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
     bag = []
     pattern_words = doc[0]
     pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

     for w in words:
          bag.append(1) if w in pattern_words else bag.append(0)

     output_row = list(output_empty)
     output_row[classes.index(doc[1])] = 1

     training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)
# create train and test lists.  X - patterns, y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print('Training data created')

# Create model with 3 layers.  First layer 128 neurons, second layer 64 neurons
# and 3rd output layer contains number of neurons equal to number of intents to
# predict
# output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model.  Stochastic gradient descent with Nesterov accelerated
# gradient gives good
# results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=1000, batch_size=5, verbose=1)
model.save(os.path.join(intents_path, 'intents_chatbot_model.h5'), hist)

print('model created')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
32 documents
5 classes ['goodbye', 'greeting', 'name', 'options', 'thanks']
42 unique lemmatized words ["'s", ',', 'about', 'anyone', 'are', 'awesome', 'bye', 'can', 'day', 'do', 'ekse', 'for', 'good', 'goodbye', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'is', 'know', 'later', 'me', 'name', 'ok', 'purpose', 'see', 'tell', 'thank', 'thanks', 'that', 'there', 'ungubani', 'what', 'whats', 'who', 'you', 'your', 'yourself']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:72: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Training data created


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/1000
7/7 [==============================] - 2s 3ms/step - loss: 1.6774 - accuracy: 0.1875
Epoch 2/1000
7/7 [==============================] - 0s 2ms/step - loss: 1.4957 - accuracy: 0.4688
Epoch 3/1000
7/7 [==============================] - 0s 2ms/step - loss: 1.4358 - accuracy: 0.4688
Epoch 4/1000
7/7 [==============================] - 0s 2ms/step - loss: 1.4199 - accuracy: 0.4062
Epoch 5/1000
7/7 [==============================] - 0s 2ms/step - loss: 1.4264 - accuracy: 0.4688
Epoch 6/1000
7/7 [==============================] - 0s 2ms/step - loss: 1.3109 - accuracy: 0.5000
Epoch 7/1000
7/7 [==============================] - 0s 2ms/step - loss: 1.2774 - accuracy: 0.5000
Epoch 8/1000
7/7 [==============================] - 0s 2ms/step - loss: 1.1585 - accuracy: 0.6250
Epoch 9/1000
7/7 [==============================] - 0s 2ms/step - loss: 1.1026 - accuracy: 0.5938
Epoch 10/1000
7/7 [==============================] - 0s 3ms/step - loss: 1.0788 - accuracy: 0.6875
Epoch 11/1000
7/7 [